In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgb
import sys

In [9]:
t1 = time.time()
df_train = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/train_onlycount_last75m.csv')

t2 = time.time()
print('training loading done! Time: {}'.format(t2 - t1))
print('size is: {}'.format(sys.getsizeof(df_train) / 1024 ** 3))
print('length is {}'.format(len(df_train)))

training loading done! Time: 37.36579656600952
size is: 6.705522634088993
length is 75000000


# Specify Categorical Columns

In [10]:
df_train.columns

Index(['ip_day_intesthh_count', 'ip_day_hour_count', 'ip_os_day_hour_count',
       'ip_app_day_hour_count', 'ip_app_os_day_hour_count',
       'app_day_hour_count', 'channel', 'app', 'os', 'device', 'hour',
       'is_attributed'],
      dtype='object')

In [11]:
categorical_col = [ 'app', 'device', 'os', 'channel', 'hour', ]
target = 'is_attributed'
feature_cols = list(set(df_train.columns) - set([target]))

In [4]:
# # remove minute and second
# categorical_col = ['app', 'device', 'os', 'channel', 'hour']
# target = 'is_attributed'
# feature_cols = list(set(df_train.columns) - set([target]) - set(['minute', 'second', 'ip']))

In [5]:
# # remove minute and second
# categorical_col = [ 'device' ]
# target = 'is_attributed'
# feature_cols = list(set(df_train.columns) - set([target]) - set(['minute', 'second', 'ip', 'app', 'os', 'channel', 'hour']))

# Get Validation -- Last 10%

In [5]:
# ratio = 0.3
# length = len(df_train)
# front = int( (1-ratio) * length)
# trainset = df_train.iloc[:front]
# valset = df_train.iloc[front:]

# print(sys.getsizeof(trainset)/ 1024 **3)
# print(len(trainset))

4.6938658729195595
52500000


# Get Validation -- Head 10%

In [14]:
ratio = 0.3
length = len(df_train)
front = int( ratio * length)

valset = df_train.iloc[:front]
trainset = df_train.iloc[front:]


print(sys.getsizeof(trainset)/ 1024 **3)
print(len(trainset))

4.69386587664485
52500000


# Get Validation -- random 10 %

In [7]:
# from sklearn.model_selection import train_test_split

# ratio = 0.1

# trainset, valset = train_test_split(df_train,test_size=ratio, random_state=19)
# print(sys.getsizeof(trainset)/ 1024 **3)
# print(len(trainset))

In [17]:
y_train = trainset[target].values
y_val = valset[target].values

lgb_train = lgb.Dataset(trainset[feature_cols], y_train, categorical_feature = categorical_col)
lgb_val = lgb.Dataset(valset[feature_cols], y_val, categorical_feature = categorical_col)

zeros = len(y_train[y_train == 0])
scale_pos_weight = len(y_train[y_train == 0]) / len(y_train) * 100
import gc
# del df_train
gc.collect()

print('train negative rate: {}'.format(scale_pos_weight))
print('val negative rate: {}'.format(len(y_val[y_val == 0]) / len(y_val) * 100))

train negative rate: 99.74658857142857
val negative rate: 99.80628444444444


# Train Lightgbm

In [18]:
params = {
        'objective': 'binary',
        'boosting': 'gbdt',
        'num_rounds': 2000,
        'learning_rate': 0.1,
        'num_leaves': 7,
        'num_threads': 20, # best speed: set to number of real cpu cores, which is vCPU/2
        'device': 'cpu',
        'max_depth': -1, # no limit. This is used to deal with over-fitting when #data is small.
        'min_data_in_leaf': 100,  #minimal number of data in one leaf. Can be used to deal with over-fitting
        'feature_fraction': 0.7, #For example, if set to 0.8, will select 80% features before training each tree.  speed up training / deal with over-fitting
        'feature_fraction_seed': 1,
        'early_stopping_round':50,
        'bagging_fraction': 0.7, #Randomly select part of data without resampling
        'bagging_freq': 1, #frequency for bagging, 0 means disable bagging. k means will perform bagging at every k iteration. to enable bagging, bagging_fraction should be set as well
        'bagging_seed': 1,
        #'max_bin': 255,
        'verbose': 0,
        'scale_pos_weight': scale_pos_weight,
        'metric' : [ 'auc']
    }

model = lgb.train(params, train_set=lgb_train, valid_sets=lgb_val, verbose_eval=10)

/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:104: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1027: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/home/kai/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:668: UserWarning: categorical_feature in param dict is overrided.
  warnings.warn('categorical_feature in param dict is overrided.')


Training until validation scores don't improve for 50 rounds.
[10]	valid_0's auc: 0.962029
[20]	valid_0's auc: 0.96534
[30]	valid_0's auc: 0.968245
[40]	valid_0's auc: 0.969725
[50]	valid_0's auc: 0.971405
[60]	valid_0's auc: 0.972605
[70]	valid_0's auc: 0.973409
[80]	valid_0's auc: 0.974173
[90]	valid_0's auc: 0.974659
[100]	valid_0's auc: 0.974987
[110]	valid_0's auc: 0.975304
[120]	valid_0's auc: 0.975601
[130]	valid_0's auc: 0.975816
[140]	valid_0's auc: 0.97596
[150]	valid_0's auc: 0.976173
[160]	valid_0's auc: 0.976265
[170]	valid_0's auc: 0.97632
[180]	valid_0's auc: 0.976416
[190]	valid_0's auc: 0.976483
[200]	valid_0's auc: 0.976509
[210]	valid_0's auc: 0.976643
[220]	valid_0's auc: 0.976678
[230]	valid_0's auc: 0.97666
[240]	valid_0's auc: 0.976719
[250]	valid_0's auc: 0.97676
[260]	valid_0's auc: 0.976747
[270]	valid_0's auc: 0.976759
[280]	valid_0's auc: 0.976796
[290]	valid_0's auc: 0.976802
[300]	valid_0's auc: 0.97683
[310]	valid_0's auc: 0.976894
[320]	valid_0's auc: 0.

In [19]:
from sklearn.metrics import roc_auc_score
pred_val = model.predict(valset[feature_cols])
print(roc_auc_score(y_val,pred_val))

0.977174661851


In [20]:
importance = pd.Series(model.feature_importance(), index=feature_cols)
importance = importance.sort_values(ascending=False)
if len(model.feature_importance()) != len(feature_cols):
    raise ValueError('Feature importance has length: {}, \n while feature number is {}'.
                     format(len(model.feature_importance()), len(feature_cols)))
importance.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/output/importance__only_count_submission0406.csv')

In [21]:
importance

channel                     985
os                          644
app                         598
hour                        318
ip_day_intesthh_count       245
ip_app_os_day_hour_count    209
ip_day_hour_count           198
app_day_hour_count          126
ip_os_day_hour_count        119
ip_app_day_hour_count       101
device                       63
dtype: int64

In [22]:
df_test = pd.read_csv('/home/kai/data/kaggle/talkingdata/wl/data/features/test_onlycount_last75m.csv')

# prediction
df_test_raw = pd.read_csv('/home/kai/data/kaggle/talkingdata/data/test.csv')
print('loading file done!')
df_test = df_test[list(valset[feature_cols].columns)]
df_sub = pd.DataFrame()
df_sub['click_id'] = df_test_raw['click_id']
df_sub['is_attributed'] = model.predict(df_test)
print('predicting file done!')
df_sub.to_csv('/home/kai/data/kaggle/talkingdata/wl/data/submission/train_onlycount_last75m_0406_1.csv.gz', compression='gzip', index=False)


In [22]:
a = ['ip_day_hour_count', 'ip', 'day', 'hour']
df_train.loc[:20,a ]

,ip_day_hour_count,ip,day,hour
0,43,143414,NaN,12
1,18,173096,NaN,12
2,42,8210,NaN,12
3,19,5746,NaN,12
4,65,31475,NaN,12
5,11,251465,NaN,12
6,164,163593,NaN,12
7,158,58288,NaN,12
8,134,27038,NaN,12
9,414,67682,NaN,12
